# Face Recognition

Trong bài tập này, ta sẽ xây dựng một hệ thống nhận dạng khuôn mặt. Nhiều ý tưởng được trình bày ở đây là từ  [FaceNet](https://arxiv.org/pdf/1503.03832.pdf), va [DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf). 

Các vấn đề nhận dạng khuôn mặt thường chia thành hai loại:

- **Face Verification** - "đây có phải là người đang yêu cầu không?". Ví dụ, tại một số sân bay, ta có thể thông qua trạm hải quan bằng cách để hệ thống quét hộ chiếu và sau đó xác minh rằng bạn (người mang hộ chiếu) có phải là người chính xác hay không. Điện thoại di động mở khóa bằng khuôn mặt cũng đang sử dụng xác minh khuôn mặt. Đây là một bài toán đối sánh 1:1

- **Face Recognition** - "người này là ai?". Đây là một bài toán đối sánh 1: K.

FaceNet học một mạng nơ-ron mã hóa hình ảnh khuôn mặt thành một vectơ gồm 128 units. Bằng cách so sánh hai vectơ như vậy, ta có thể xác định xem hai bức ảnh có phải của cùng một người hay không.
    
**Trong bài vấn đề này ta sẽ đưa ra các bước thực hiện sau:**
- Thực hiện triplet loss function
- Sử dụng mô hình được đào tạo trước để ánh xạ hình ảnh khuôn mặt thành các encoding 128 chiều
- Sử dụng các mã hóa này để thực hiện xác minh khuôn mặt và nhận dạng khuôn mặt

#### Load packages
Hãy load các thư viện cần thiết.

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


## 0 - Xác minh khuôn mặt

Trong Xác minh khuôn mặt, bạn được cung cấp hai hình ảnh và bạn phải xác định xem chúng có phải của cùng một người hay không. Cách đơn giản nhất để làm điều này là so sánh hai hình ảnh theo từng pixel. Nếu khoảng cách giữa hai hình ảnh nhỏ hơn một ngưỡng đã chọn, đó có thể là cùng một người!

<img src="images/pixel_comparison.png" style="width:380px;height:150px;">
<caption><center> <u> <font color='purple'> Figure 1 </u></center></caption>

* Tất nhiên, thuật toán này hoạt động thực sự kém, vì các giá trị pixel thay đổi đáng kể do sự thay đổi về ánh sáng, hướng khuôn mặt của người đó, thậm chí cả những thay đổi nhỏ ở vị trí đầu, v.v.
* Ta sẽ thấy rằng thay vì sử dụng hình ảnh raw, ta có thể học cách mã hóa, $f(img)$.  
* Bằng cách sử dụng encoding cho mỗi hình ảnh, phép so sánh element-wise sẽ đưa ra dự đoán chính xác hơn về việc liệu hai hình ảnh có phải của cùng một người hay không.

## 1 - Mã hóa hình ảnh khuôn mặt thành một vector 128 chiều

### 1.1 - Sử dụng ConvNet để tính toán các mã hóa

Mô hình FaceNet cần rất nhiều dữ liệu và thời gian đào tạo lâu dài. Vì vậy, một cách người ta thường làm trong học sâu là tải trọng số và load model đã được đào tạo trước. Kiến trúc mạng tuân theo mô hình tại Figure 3 trong paper [Szegedy *et al.*](https://arxiv.org/abs/1409.4842). Kiến trúc model ta có thể xem trong file `inception_blocks_v2.py`

Những điều quan trọng bạn cần biết là:

- Mạng này sử dụng hình ảnh RGB 96x96 chiều làm đầu vào. Cụ thể, nhập hình ảnh khuôn mặt (hoặc một batch hình ảnh khuôn mặt $m$) dưới dạng một khối $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$ 
- Nó xuất ra một ma trận có dạng $(m, 128)$ mã hóa từng hình ảnh khuôn mặt đầu vào thành một vectơ 128 chiều

Mã sau tạo model cho hình ảnh khuôn mặt.

In [3]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [5]:
FRmodel.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 96, 96)     0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 3, 102, 102)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 64, 48, 48)    9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn1 (BatchNormalization)         (None, 64, 48, 48)    256         conv1[0][0]                      
___________________________________________________________________________________________

Bằng cách sử dụng lớp kết nối đầy đủ 128 nơ-ron làm lớp cuối cùng , mô hình đảm bảo rằng đầu ra là một vectơ mã hóa có kích thước 128. Sau đó, ta sử dụng các mã hóa để so sánh hai hình ảnh khuôn mặt như sau:

<img src="images/distance_kiank.png" style="width:680px;height:250px;">

Vì vậy, một encoding tốt nếu:
- Các encoding của hai hình ảnh của cùng một người khá giống nhau.
- Các encoding của hai hình ảnh của những người khác nhau rất khác nhau.

Triplet loss function  sẽ là hàm loss function và cố gắng "đẩy" mã hóa của hai hình ảnh của cùng một người (Anchor và Positive) gần nhau hơn, đồng thời "kéo" các mã hóa của hai hình ảnh của những người khác nhau (Anchor, Negative) xa nhau hơn. 


### 1.2 -Triplet Loss function

Đối với hình ảnh $x$, chúng ta ký hiệu mã hóa của nó là $f(x)$, trong đó $f$ là hàm được tính toán bởi mạng nơ-ron.

<img src="images/f_x.png" style="width:380px;height:150px;">

<!--
We will also add a normalization step at the end of our model so that $\mid \mid f(x) \mid \mid_2 = 1$ (means the vector of encoding should be of norm 1).
!-->

Training sẽ sử dụng triplets of images $(A, P, N)$:  

- A là "Anchor" image--bức ảnh của một người 
- P là "Positive" image--bức ảnh của cùng một người giống A
- N là "Negative" image--bức ảnh của một người khác A

Bộ ba ảnh này được chọn từ tập dữ liệu đào tạo của chúng ta. Chúng ta sẽ viết $ (A ^ {(i)}, P ^ {(i)}, N ^ {(i)}) $ để biểu thị ví dụ đào tạo $ i $ -th.

Ta muốn chắc chắn rằng hình ảnh  $A^{(i)}$  gần với ảnh $P^{(i)}$ hơn ảnh $N^{(i)}$):

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

Do đố ta sẽ tối ưu hóa hàm "triplet cost":

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

Ta sử dụng kí hiệu "$[z]_+$" có nghĩa là $max(z,0)$.  

Notes:
- ta muốn (1) nhỏ.
- Và muốn (2) là khoảng cách bình phương giữa "A" và "N" tương đối lớn.
- $\alpha$ được gọi là ngưỡng. Nó là một hyperparameter mà ta chọn theo cách thủ công. Chúng ta sẽ sử dụng $\alpha = 0.2$. 

In [6]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Thực hiện triplet_loss như được xác định bởi công thức(3)
    
    Arguments:
    y_true -- true labels
    y_pred -- danh sách python chứa ba đối tượng:
            anchor -- encodings cho hình ảnh anchor có shape (None, 128)
            positive -- encodings cho hình ảnh positive có shape (None, 128)
            negative -- encodings cho hình ảnh negative có shape (None, 128)
    
    Returns:
    loss -- số thực
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Step 1: Tính toán khoảng cách (encoding) giữa A và P
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    # Step 2: Tính toán khoảng cách (encoding) giữa A và N
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    # Step 3: trừ hai khoảng cách trước đó và cộng với alpha.
    basic_loss = tf.add((tf.subtract(pos_dist,neg_dist)),alpha)
    # Step 4: Lấy max(basic_loss,0)
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.0))

    return loss

## 2 - Load model được đào tạo trước

FaceNet được đào tạo bằng cách tối ưu hóa triplet loss function. Nhưng vì quá trình đào tạo đòi hỏi nhiều dữ liệu và tính toán nhiều đồng nghĩa với máy tính hiện đại để train nên ta sẽ không đào tạo lại từ đầu. Thay vào đó,load một mô hình đã được đào tạo trước đó.

In [7]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

Ví dụ về khoảng cách giữa các encoding:

<img src="images/distance_matrix.png" style="width:380px;height:200px;">
<br>

Tiếp theo ta hãy sử dụng mô hình này để thực hiện xác minh khuôn mặt và nhận dạng khuôn mặt!

## 3 - Áp dụng model

Giả sử ta đang xây dựng một hệ thống cho một tòa nhà văn phòng và người quản lý tòa nhà muốn cung hệ thống nhận dạng khuôn mặt để cho phép nhân viên ra vào tòa nhà.

Ta muốn xây dựng hệ thống **Xác minh khuôn mặt** cho phép truy cập vào danh sách những nhân viên có trong database của tòa nhà. Để được nhận vào, mỗi người phải quẹt thẻ ID (thẻ căn cước) để nhận dạng mình ở cửa ra vào. Sau đó, hệ thống xác minh khuôn mặt sẽ kiểm tra xem họ có phải là nhân viên trong tòa nhà hay không.

### 3.1 - Xác minh khuôn mặt

Hãy xây dựng một cơ sở dữ liệu chứa một vectơ mã hóa cho mỗi người được phép vào văn phòng. Để tạo encoding, chúng ta sử dụng ```img_to_encoding(image_path, model)```

Mã sau sẽ xây dựng database (được biểu diễn dưới dạng python dict). Database này ánh xạ  mỗi người thành encoding 128 chiều trên khuôn mặt của họ.

In [26]:
img_test  = cv2.imread("images/hieule.jpeg")
img_test_resized = cv2.resize(src=img_test, dsize=(96, 96))
plt.imsave("images/hieu_test_resize.jpg",img_test_resized)

In [27]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["Hieu Le"] = img_to_encoding("images/hieu_test_resize.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

Bây giờ, khi ai đó xuất hiện ở cửa trước và quẹt thẻ ID của họ (do đó cung cấp cho ta tên của họ), ta có thể tra cứu encoding của họ trong database và sử dụng nó để kiểm tra xem người đứng ở cửa trước có khớp với tên trên không ID.

Mã sau thực hiện từng bước như sau:

1. Tính toán mã hóa của hình ảnh từ `image_path`.
2. Tính toán khoảng cách giữa encoding này và encoding hình ảnh nhận dạng được lưu trữ trong database.
3. Mở cửa nếu khoảng cách nhỏ hơn ngưỡng đặt ra ở đây là 0,7, nếu không thì không được mở.


In [25]:
def verify(image_path, identity, database, model):
    """
    Hàm xác minh xem người trên hình ảnh "image_path" có phải là "identity" hay không.
    
    Arguments:
    image_path -- hình ảnh cần xác minh
    identity -- tên của người bạn muốn xác minh danh tính. Phải là nhân viên làm việc văn phòng.
    database -- từ điển python ánh xạ tên của tên người được phép với mã hóa encoding của họ.
    model -- model ta đã load
    
    Returns:
    dist -- khoảng cách giữa image_path và hình ảnh của "identity" trong database.
    door_open -- True nếu cùng một người, False nếu otherwise
    """
    
    ### START CODE HERE ###
    
    # Step 1: Tính toán encoding cho hình ảnh.
    encoding = img_to_encoding(image_path,model)
    
    # Step 2: Tính toán khoảng cách
    dist = np.linalg.norm(encoding - database[identity])
    
    # Step 3: Mở cửa nếu dist < 0.7, else không mở cửa
    if dist<0.7:
        print("Đây là " + str(identity) + ", Chào mừng bạn")
        door_open = True
    else:
        print("Đây không phải là " + str(identity) + ", Bạn không phải nhân viên công ty")
        door_open = False
        
    return dist, door_open

Younes đang cố gắng vào văn phòng và máy ảnh chụp ảnh anh ta ("images / camera_0.jpg"). Hãy chạy thuật toán xác minh của bạn trên hình này:
<img src="images/camera_0.jpg" style="width:100px;height:100px;">

In [28]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

Đây là younes, Chào mừng bạn


(0.65939289, True)

Tôi, người không làm việc trong văn phòng, đã lấy trộm thẻ ID của Younes và cố gắng vào văn phòng. Máy ảnh đã chụp ảnh Tôi ("images/hieu_test_resize.jpg"). Hãy chạy thuật toán xác minh để kiểm tra xem tôi có thể vào được không.
<img src="images/hieu_test_resize.jpg" style="width:100px;height:100px;">

In [29]:
verify("images/hieu_test_resize.jpg", "younes", database, FRmodel)

Đây không phải là younes, Bạn không phải nhân viên công ty


(0.91193688, False)

### 3.2 - Face Recognition

Hệ thống xác minh khuôn mặt của ta hầu hết hoạt động tốt. Nhưng vì Younes bị đánh cắp thẻ căn cước, khi anh ta quay lại văn phòng vào ngày hôm sau và không thể vào được!

Để giải quyết vấn đề này, ta muốn thay đổi hệ thống xác minh khuôn mặt của mình thành hệ thống nhận dạng khuôn mặt. Như vậy, không ai phải mang theo thẻ căn cước nữa.

Bạn sẽ triển khai hệ thống nhận dạng khuôn mặt lấy hình ảnh đầu vào và xác định xem đó có phải là một trong những người được ủy quyền hay không (và nếu có thì là ai). Không giống như hệ thống xác minh khuôn mặt trước đây, chúng tôi sẽ không còn lấy tên của một người làm một trong những đầu vào nữa.

Hàm sau đây thực hiện hệ thông nhận dạng khuôn mặt theo 2 bước:

1. Tính toán encoding của hình ảnh từ image_path
2. Tìm encoding từ database có khoảng cách nhỏ nhất với mã hóa target.

In [20]:
def who_is_it(image_path, database, model):
    """
    Triển khai nhận dạng khuôn mặt bằng cách tìm ai là người trên hình ảnh image_path.
    
    Arguments:
    image_path -- Hình ảnh cần nhận dạng
    database -- cơ sở dữ liệu chứa hình ảnh encoding cùng với tên của người trên hình ảnh
    model -- Model
    
    Returns:
    min_dist -- khoảng cách tối thiểu giữa encoding image_path và các encoding từ database
    identity -- dự đoán tên cho người trên image_path
    """
       
    ## Step 1: Tính toán "encoding" target cho hình ảnh.
    encoding = img_to_encoding(image_path,model)
    
    ## Step 2: Tìm encoding gần nhất
    min_dist = 100
    for (name, db_enc) in database.items():
        
        # Tính toán khoảng cách giữa "encoing" target và db_enc hiện tại từ database.
        dist = np.linalg.norm(encoding - db_enc)

        # Nếu khoảng cách này nhỏ hơn min_dist, thì hãy đặt min_dist thành dist và identity thành tên.
        if dist<min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        print("Không có trong database, bạn không được vào.")
    else:
        print ("Đây là " + str(identity) + ", khoảng cách: " + str(min_dist))
        
    return min_dist, identity

Younes is at the front-door and the camera takes a picture of him ("images/camera_0.jpg"). Let's see if your who_it_is() algorithm identifies Younes. 

In [21]:
who_is_it("images/camera_0.jpg", database, FRmodel)

Đây là younes, khoảng cách: 0.659393


(0.65939289, 'younes')

In [30]:
who_is_it("images/hieu_test_resize.jpg", database, FRmodel)

Đây là Hieu Le, khoảng cách: 0.0


(0.0, 'Hieu Le')

Hệ thống nhận dạng khuôn mặt của ta đang hoạt động tốt! Nó chỉ cho phép những người là nhân viên qua và mọi người không cần phải mang theo thẻ ID nữa!



#### Các cách cải thiện mô hình nhận dạng khuôn mặt 

- Đưa thêm hình ảnh của từng người (trong các điều kiện ánh sáng khác nhau, chụp vào các ngày khác nhau, v.v.) vào database. Sau đó, đưa ra một hình ảnh mới, so sánh khuôn mặt mới với nhiều hình ảnh của người đó. Điều này sẽ làm tăng độ chính xác.
- Cắt hình ảnh để chỉ chứa khuôn mặt và ít vùng "background" xung quanh khuôn mặt. Việc xử lý trước này sẽ loại bỏ một số pixel không liên quan xung quanh khuôn mặt và cũng làm cho thuật toán mạnh mẽ hơn.

### Tham khảo:

- [FaceNet](https://arxiv.org/pdf/1503.03832.pdf)
- [DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf) 
- https://github.com/iwantooxxoox/Keras-OpenFace.
- https://github.com/davidsandberg/facenet 
